<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to PoP iMOD Sim HD outputs
</span>

You're advised to run the notebook cells one by one to understand what they're doing and avoid any problems.

In [1]:
from WS_Mdl import utils as Utl
from WS_Mdl import utils_imod as Utl_IM
from WS_Mdl import geo
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# Options

This block can be replaced with a configuration excel spreadsheet that will be read as a DF.
I think it's preferable to have a working script first, then convert it to functions and a shorter main script, that's why I'm not using the Cfg s/s o the get go.

In [2]:
MdlN_S = 'NBr13'
SmB = True

# Infer parameters

In [3]:
d_paths = Utl.get_MdlN_paths(MdlN_S)
d_INI = Utl.INI_to_d(d_paths['path_INI_S'])
Mdl = ''.join([i for i in MdlN_S if i.isalpha()])
MdlN_B, path_PoP, path_MdlN_S, path_MdlN_B = [ d_paths[v] for v in ['MdlN_B', 'path_PoP', 'path_MdlN', 'path_MdlN_B'] ]

In [4]:
l_Mo_winter = [10, 11, 12, 1, 2, 3]
l_Mo_summer = [i for i in range(1,13) if i not in l_Mo_winter]
l_layers = ['L1']

In [5]:
S_Fi_HD_B = pd.Series([i for i in os.listdir(os.path.join(path_MdlN_B, f'GWF_1/MODELOUTPUT/HEAD/HEAD')) if i.split('.')[0].split('_')[-1] in l_layers])
S_Fi_HD_S = pd.Series([i for i in os.listdir(os.path.join(path_MdlN_S, f'GWF_1/MODELOUTPUT/HEAD/HEAD')) if i.split('.')[0].split('_')[-1] in l_layers])

S_path_HD_B = pd.Series([os.path.join(path_MdlN_B, f'GWF_1/MODELOUTPUT/HEAD/HEAD', i) for i in S_Fi_HD_B])
S_path_HD_S = pd.Series([os.path.join(path_MdlN_S, f'GWF_1/MODELOUTPUT/HEAD/HEAD', i) for i in S_Fi_HD_S])

In [6]:
S_Fi_B_Mo = S_Fi_HD_B.str.split("_").str[1].str[4:6].astype(int)
S_Fi_S_Mo = S_Fi_HD_S.str.split("_").str[1].str[4:6].astype(int)

# Load IDF's and calculate AVGs

## B

In [7]:
DF_IDF_B = Utl_IM.IDFs_to_DF(S_path_HD_B)

Processing .IDF files: 100%|██████████| 3287/3287 [01:19<00:00, 41.18it/s]


In [ ]:
A_B_AVG = np.mean(np.stack(DF_IDF_B['IDF'].apply(lambda x: x[0])), axis=0)
A_B_AVG_summer = np.mean(np.stack(DF_IDF_B.loc[ S_Fi_B_Mo.isin(l_Mo_summer), 'IDF'].apply(lambda x: x[0])), axis=0)
A_B_AVG_winter = np.mean(np.stack(DF_IDF_B.loc[ S_Fi_B_Mo.isin(l_Mo_winter), 'IDF'].apply(lambda x: x[0])), axis=0)
d_A_B_AVG = {'full_year': A_B_AVG,
             'summer': A_B_AVG_summer,
             'winter': A_B_AVG_winter}

In [ ]:
IDF_MtDt = DF_IDF_B.at[0, 'IDF'][1] # The metadata for all IDF's is the same, except for time. But time doesn't matter for what we'll do.

In [ ]:
for k in d_A_B_AVG:
    group = '' if k=='full_year' else (k+'_')
    path_Out = os.path.join(path_PoP, 'HD_map', f'{Mdl}{SimN_B}', f'HD_{group}{Mdl}{SimN_B}.')
    os.makedirs(os.path.dirname(path_Out), exist_ok=True)
    A_to_Raster_n_IDF(d_A_B_AVG[k], IDF_MtDt, path_Out)

## S

In [ ]:
DF_IDF_S = make_DF_IDF(path_S, S_Fi_S)

In [ ]:
A_S_AVG = np.mean(np.stack(DF_IDF_S['IDF'].apply(lambda x: x[0])), axis=0)
A_S_AVG_summer = np.mean(np.stack(DF_IDF_S.loc[ S_Fi_S_Mo.isin(l_Mo_summer), 'IDF'].apply(lambda x: x[0])), axis=0)
A_S_AVG_winter = np.mean(np.stack(DF_IDF_S.loc[ S_Fi_S_Mo.isin(l_Mo_winter), 'IDF'].apply(lambda x: x[0])), axis=0)
d_A_S_AVG = {'full_year': A_S_AVG,
             'summer': A_S_AVG_summer,
             'winter': A_S_AVG_winter}

In [ ]:
IDF_MtDt = DF_IDF_S.at[0, 'IDF'][1] # The metadata for all IDF's is the same, except for time. But time doesn't matter for what we'll do.

In [ ]:
for k in d_A_S_AVG:
    group = '' if k=='full_year' else (k+'_')
    path_Out = os.path.join(path_PoP, 'HD_map', f'{Mdl}{SimN_S}', f'HD_{group}{Mdl}{SimN_S}.')
    os.makedirs(os.path.dirname(path_Out), exist_ok=True)
    A_to_Raster_n_IDF(d_A_S_AVG[k], IDF_MtDt, path_Out)

In [ ]:
# import shutil
# pd.set_option('display.width', 200)
# pd.set_option('display.max_colwidth', None)
# import openpyxl as xl
# import matplotlib.pyplot as plt
# import geopandas as gpd
# import rasterio
# from rasterio.transform import from_bounds
# from shapely.geometry import shape
# import xarray as xr